<a href="https://colab.research.google.com/github/saciva-team-2b/Saciva-University-Recommendation-System-2B/blob/main/Saciva_University_Recommendation_System_2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Saciva University Recommendation System 2B**



---


**Resources and Documents:**  
[Trello Board](https://trello.com/b/nXdTl8OF/saciva-university-recommendation-system-2b) | [Google Drive](https://drive.google.com/drive/folders/1Psq7fGf3CNVNYAeFybnAVz_SMj8Py3TY) | [Github Repo](https://github.com/saciva-team-2b/Saciva-University-Recommendation-System-2B)

---

### **Challenge Summary**
Suggest top 5 Higher Education universities to a student by matching student's with the universities' profile. Student Profile: Current Education level, Field of study, Chosen research, entrepreneurship, and financial status. University Profile: location (climate, cost of living), the field, overall ranking, fees and tuition, other factors.

### **Context / Impact**
Saciva is an app for Students, especially international students to help through their higher education journey and choosing a university is the first step and often research on academics and tuition and over look factors such as climate, opportunities after in their field, cost of living, safety, etc. before they take admission.

### **Suggested Approach**
The student's profile can be vectorized to find the nearest data (unsupervised learning) to suggest the 5 most suitable universities to the student.

## **Data Exploration and Analysis**



1. World University Rankings (2019)
2. College Scorecard (2024)
3. College Scorecard Most Recent Institution-Level Data (2024)
4. College Scorecard Most Recent Data by Field of Study (2024)
5. Campus Safety and Security (2023)
6. Cost of Living Index (2021)

In [ ]:
import pandas as pd

pd.read_csv('/datasets/WorldUniversityRankings2019/timesData.csv')

Next, we shall import some standard ML libraries, and import more as we need.



---


### GitHub Commands
